# Hoja de Trabajo 2

- Josue Marroquin 22397
- Sebastian Huertas 22295

In [7]:
# Libs
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## Ejercicio 1 - Experimentación Práctica

### Task 1 - Preparación del conjunto de datos
Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos
en conjuntos de entrenamiento y validación.

In [5]:
# Cargar el conjunto de datos de Iris
iris = load_iris()
X = iris.data  # Características: sepal length, sepal width, petal length, petal width
y = iris.target  # Etiquetas: 0=setosa, 1=versicolor, 2=virginica

# Crear DataFrame para mejor visualización
feature_names = iris.feature_names
target_names = iris.target_names

df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df['species'] = df['target'].map({0: target_names[0], 1: target_names[1], 2: target_names[2]})

print("Información del conjunto de datos:")
print(f"Forma del dataset: {X.shape}")
print(f"Número de características: {X.shape[1]}")
print(f"Número de muestras: {X.shape[0]}")
print(f"Clases: {target_names}")
print(f"Distribución de clases: {np.bincount(y)}")

# Mostrar las primeras filas
print("\nPrimeras 5 filas del dataset:")
print(df.head())

# Dividir el conjunto de datos en entrenamiento y validación (80% - 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Mantener la proporción de clases en ambos conjuntos
)

print(f"\nConjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Conjunto de validación: {X_val.shape[0]} muestras")
print(f"Distribución en entrenamiento: {np.bincount(y_train)}")
print(f"Distribución en validación: {np.bincount(y_val)}")

# Estandarizar las características (opcional pero recomendado para muchos algoritmos)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print("\nDatos preparados exitosamente para entrenamiento y validación")
print("Variables disponibles:")
print("- X_train, X_val: características originales")
print("- X_train_scaled, X_val_scaled: características estandarizadas")
print("- y_train, y_val: etiquetas")
print("- df: DataFrame completo con información descriptiva")

Información del conjunto de datos:
Forma del dataset: (150, 4)
Número de características: 4
Número de muestras: 150
Clases: ['setosa' 'versicolor' 'virginica']
Distribución de clases: [50 50 50]

Primeras 5 filas del dataset:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target species  
0       0  setosa  
1       0  setosa  
2       0  setosa  
3       0  setosa  
4       0  setosa  

Conjunto de entrenamiento: 120 muestras
Conjunto de validación: 30 muestras
Distribución en entrenamiento: [40 40 40]
Distribución en validación: [10 10 10]

Datos preparados exi

### Task 2 - Arquitectura modelo
Cree una red neuronal feedforward simple utilizando nn.Module de PyTorch. Luego, defina capa de entrada, capas
ocultas y capa de salida. Después, elija las funciones de activación y el número de neuronas por capa

In [8]:
# Task 2 - Arquitectura del modelo

# Configurar dispositivo (GPU si está disponible, sino CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo utilizado: {device}")

# Definir la arquitectura de la red neuronal feedforward
class IrisClassifier(nn.Module):
    def __init__(self, input_size=4, hidden_size1=16, hidden_size2=8, num_classes=3):
        super(IrisClassifier, self).__init__()
        
        # Definir las capas
        self.fc1 = nn.Linear(input_size, hidden_size1)      # Capa de entrada: 4 -> 16
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)    # Primera capa oculta: 16 -> 8
        self.fc3 = nn.Linear(hidden_size2, num_classes)     # Capa de salida: 8 -> 3
        
        # Capa de dropout para regularización
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        # Forward pass a través de la red
        x = F.relu(self.fc1(x))         # Activación ReLU en primera capa
        x = self.dropout(x)             # Aplicar dropout
        x = F.relu(self.fc2(x))         # Activación ReLU en segunda capa
        x = self.dropout(x)             # Aplicar dropout
        x = self.fc3(x)                 # Capa de salida (sin activación, se aplica en loss)
        return x

# Crear instancia del modelo
model = IrisClassifier(input_size=4, hidden_size1=16, hidden_size2=8, num_classes=3)
model = model.to(device)

# Mostrar la arquitectura del modelo
print("Arquitectura del modelo:")
print(model)
print(f"\nNúmero total de parámetros: {sum(p.numel() for p in model.parameters())}")
print(f"Parámetros entrenables: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# Convertir datos de numpy a tensores de PyTorch
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train).to(device)
y_val_tensor = torch.LongTensor(y_val).to(device)

print(f"\nForma de los tensores:")
print(f"X_train: {X_train_tensor.shape}")
print(f"X_val: {X_val_tensor.shape}")
print(f"y_train: {y_train_tensor.shape}")
print(f"y_val: {y_val_tensor.shape}")

# Crear DataLoaders para entrenamiento por lotes
batch_size = 16
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"\nDataLoaders creados:")
print(f"Tamaño del lote: {batch_size}")
print(f"Número de lotes de entrenamiento: {len(train_loader)}")
print(f"Número de lotes de validación: {len(val_loader)}")

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()  # Para clasificación multiclase
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

print(f"\nConfiguración de entrenamiento:")
print(f"Función de pérdida: {criterion}")
print(f"Optimizador: {optimizer}")
print(f"Tasa de aprendizaje: 0.001")

# Función para probar el modelo con datos de ejemplo
def test_forward_pass():
    model.eval()
    with torch.no_grad():
        # Tomar una muestra pequeña para probar
        sample_input = X_train_tensor[:3]  # 3 muestras
        output = model(sample_input)
        probabilities = F.softmax(output, dim=1)
        predictions = torch.argmax(output, dim=1)
        
        print(f"\nPrueba del forward pass:")
        print(f"Input shape: {sample_input.shape}")
        print(f"Output shape: {output.shape}")
        print(f"Predictions: {predictions.cpu().numpy()}")
        print(f"Probabilities:\n{probabilities.cpu().numpy()}")

test_forward_pass()
print("\nModelo creado y configurado")

Dispositivo utilizado: cpu
Arquitectura del modelo:
IrisClassifier(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

Número total de parámetros: 243
Parámetros entrenables: 243

Forma de los tensores:
X_train: torch.Size([120, 4])
X_val: torch.Size([30, 4])
y_train: torch.Size([120])
y_val: torch.Size([30])

DataLoaders creados:
Tamaño del lote: 16
Número de lotes de entrenamiento: 8
Número de lotes de validación: 2

Configuración de entrenamiento:
Función de pérdida: CrossEntropyLoss()
Optimizador: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.0001
)
Tasa de aprendizaje: 0.001

Prueba del forward pass:
Input sha

### Task 3 - Funciones de Pérdida
Utilice diferentes funciones de pérdida comunes como Cross-Entropy Loss y MSE para clasificación. Entrene el
modelo con diferentes funciones de pérdida y registre las pérdidas de entrenamiento y test. Debe utilizar al menos 3
diferentes funciones. Es decir, procure que su código sea capaz de parametrizar el uso de diferentes funciones de
pérdida.